In [1]:
import pandas as pd
import re
import json
import requests
import folium
from folium.plugins import HeatMap, HeatMapWithTime

In [298]:
data_20_23 = pd.read_csv('C:/Users/sjhty/Downloads/monthly_passengers_20_23.csv')
data_10_16 = pd.read_csv('C:/Users/sjhty/Downloads/monthly_passengers_10_16.csv')
data_2017 = pd.read_excel('C:/Users/sjhty/Downloads/서울교통공사 2017년 일별 역별 시간대별 승하차인원(1_8호선).xlsx', header=1)
data_2018 = pd.read_excel('C:/Users/sjhty/Downloads/서울교통공사 2018년 일별 역별 시간대별 승하차인원(1_8호선).xlsx', header=1)
data_2019 = pd.read_excel('C:/Users/sjhty/Downloads/서울교통공사 2019년 일별 역별 시간대별 승하차인원(1_8호선).xlsx', header=1)

In [299]:
data_2019['날짜'] = data_2019['날짜'].apply(lambda n:n.strftime('%Y-%m'))
data_2019 = data_2019[['역명', '날짜', '합 계']].groupby(['역명', '날짜']).sum().reset_index()
data_2018['날짜'] = data_2018['날짜'].apply(lambda n:n.strftime('%Y-%m'))
data_2018 = data_2018[['역명', '날짜', '합 계']].groupby(['역명', '날짜']).sum().reset_index()
data_2017['날짜'] = data_2017['날짜'].apply(lambda n:pd.to_datetime(n).strftime('%Y-%m'))
data_2017 = data_2017[['역명', '날짜', '합 계']].groupby(['역명', '날짜']).sum().reset_index()

data_17_19 = pd.concat([data_2017, data_2018, data_2019], axis=0)
data_17_19.columns = ['역명', '연월', '총 승하차 인원']

In [300]:
data_10_16['역명'] = data_10_16['역명'].apply(lambda n:n.split('(')[0])
data_17_19['역명'] = data_17_19['역명'].apply(lambda n:n.split('(')[0])
data_20_23['역명'] = data_20_23['역명'].apply(lambda n:n.split('(')[0])

In [301]:
data_10_23 = pd.concat([data_10_16, data_17_19, data_20_23], axis=0)

In [302]:
data_10_23['역명'] = data_10_23['역명'].apply(lambda n:re.sub('동대문역사문화공원5', '동대문역사문화공원', n))
data_10_23['역명'] = data_10_23['역명'].apply(lambda n:re.sub('미아삼거리', '미아사거리', n))


## 지하철 위도, 경도 정보 가져오기

In [234]:
auth_key = '70e4b8cf-72cf-48fd-ad64-5230451fa045'
url = 'http://t-data.seoul.go.kr/apig/apiman-gateway/tapi/TaimsKsccDvSubwayStationGeom/1.0'

params = {
    'apikey': auth_key,
    'startRow': '1',
    'rowCnt': '1000'
}

response = requests.get(url, params=params)
data = response.json()

#### 같은 역 이름으로 호선이 다른 경우 좌표가 두 개 생김, 향후 사용한다면 해당 부분 수정 필요

In [305]:
name = []
longitude = []
latitude = []

for i in data:
    name.append(i['stnKrNm'])
    longitude.append(i['convX'])
    latitude.append(i['convY'])
    
sb_location = pd.DataFrame(
    {'name':name,
    'longitude':longitude,
    'latitude':latitude}
)

### 지하철 유동인구에 따른 가게들의 오픈, 폐업, 현재 엽엉중을 살펴보자

In [318]:
sb_location['name'] = sb_location['name'].apply(lambda n:n.split('(')[0])
sb_location.drop_duplicates(subset=['name'], keep='first', inplace=True)
sb_data_lc = pd.merge(data_10_23, sb_location, how='left', left_on='역명', right_on='name').drop(['name'], axis=1)

```sql
SELECT date
  , ARRAY_AGG(name) AS name
  , ARRAY_AGG(longitude) AS longitude
  , ARRAY_AGG(latitude) AS latitude
FROM `temp.sb_data`
GROUP BY date
```

In [155]:
st_data_lc = pd.read_csv('sb_data_1차.csv')

In [182]:
df = pd.read_csv('C:/Users/sjhty/Desktop/SHoon/gcp/pjt/store_keyword.csv')

In [184]:
df['store_name'] = df['store_name'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)
df['total_location'] = df['total_location'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)
df['normal_location'] = df['normal_location'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)
df['close_location'] = df['close_location'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)
df['open_date'] = df['open_date'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)
df['close_date'] = df['close_date'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)

In [183]:
st_data_lc['name'] = st_data_lc['name'].apply(lambda n: n.strip('[]').split(',') if n != '[,]' and n != '[]' else None)
st_data_lc['longitude'] = st_data_lc['longitude'].apply(lambda n: [float(i) for i in n.strip('[]').split(',') if n not in ['[],', '[]']])
st_data_lc['latitude'] = st_data_lc['latitude'].apply(lambda n: [float(i) for i in n.strip('[]').split(',') if n not in ['[],', '[]']])
st_data_lc['population'] = st_data_lc['population'].apply(lambda n: [float(i) for i in n.strip('[]').split(',') if n not in ['[],', '[]']])
st_data_lc['normalized_population'] = st_data_lc['normalized_population'].apply(lambda n: [float(i) for i in n.strip('[]').split(',') if n not in ['[],', '[]']])

AttributeError: 'list' object has no attribute 'strip'

In [178]:

# Folium 지도 생성
m = folium.Map(location=[37.55, 127], zoom_start=12)

# 승하차 인원을 기준으로 HeatMap 그리기
heat_data = []
for idx, row in st_data_lc.iterrows():
    heat_data.append(list(zip(row['latitude'], row['longitude'], row['normalized_population'])))
    
for i in range(len(heat_data)):
    for j in range(len(heat_data[i])):
        heat_data[i][j] = list(heat_data[i][j])

hm = HeatMapWithTime(heat_data, index=st_data_lc['date'].tolist(), auto_play=True, max_opacity=1, blur=0.05, radius=15, max_speed=60)
hm.add_to(m)

In [189]:
gongcha = df[df['keyword']=='공차'].reset_index(drop=True)

In [202]:
gongcha

,keyword,year,month,store_name,total_location,normal_location,close_location,open_date,close_date,store_count,count_normal,count_closed
0,공차,2009,4,"[공차 광운대역점, 공차파리공원점]","[127.05936664724977 37.623410209589352, 126.87...",None,"[127.05936664724977 37.623410209589352, 126.87...","[2009-04-06, 2009-04-23]","[2017-11-20, 2018-05-10]",2,0,2
1,공차,2009,5,[공차],[127.03171229271769 37.501021849866362],[127.03171229271769 37.501021849866362],None,[2009-05-08],[2023-11-21],1,1,0
2,공차,2009,6,[공차],[127.02006180081179 37.511829666777395],None,[127.02006180081179 37.511829666777395],[2009-06-29],[2021-09-07],1,0,1
3,공차,2009,7,[공차제주일도점],[126.53854963354199 33.505409807534285],[126.53854963354199 33.505409807534285],None,[2009-07-14],[2023-11-21],1,1,0
4,공차,2009,8,[해초공차6.0],[126.92625389083427 37.601778474537177],None,[126.92625389083427 37.601778474537177],[2009-08-12],[2015-05-29],1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
151,공차,2023,6,"[공차구미문성점, 공차 부산해운대구청점, 공차 롯데월드점, 공차 염창역점, 공차 원...","[128.33729583516805 36.159272682520545, 129.15...","[128.33729583516805 36.159272682520545, 129.15...","[, , , , , , , , , ]","[2023-06-09, 2023-06-09, 2023-06-01, 2023-06-2...","[2023-11-21, 2023-11-21, 2023-11-21, 2023-11-2...",10,10,0
152,공차,2023,7,"[공차 강동구청점, 공차 의정부고산점, 공차 신세계아울렛 부산점, 공차롯데마트당진점...","[127.11975503748195 37.529733691730883, 127.10...","[127.11975503748195 37.529733691730883, 127.10...","[, , , , , , ]","[2023-07-12, 2023-07-19, 2023-07-20, 2023-07-1...","[2023-11-21, 2023-11-21, 2023-11-21, 2023-11-2...",7,7,0
153,공차,2023,8,"[공차 김포구래중앙점, 공차 서울대점, 공차 반포삼호가든점]","[126.62335993899445 37.643508905126389, 126.95...","[126.62335993899445 37.643508905126389, 126.95...","[, , ]","[2023-08-11, 2023-08-30, 2023-08-29]","[2023-11-21, 2023-11-21, 2023-11-21]",3,3,0
154,공차,2023,9,"[공차 대전충남대점, 공차 홍제역점, 공차 나주혁신점, 공차 일산벨라시타점, 공차 ...","[127.3449081843261 36.362678346601406, 126.942...","[127.3449081843261 36.362678346601406, 126.942...","[, , , , , , , , ]","[2023-09-01, 2023-09-01, 2023-09-07, 2023-09-0...","[2023-11-21, 2023-11-21, 2023-11-21, 2023-11-2...",9,9,0


In [201]:
for item in gongcha:
    lat, lon = item['total_location']
    store_names = ', '.join(item['store_name'])
    folium.Marker(
        location=[lat, lon],
        popup=f"{store_names}<br>Open Date: {item['open_date']}<br>Close Date: {item['close_date']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

TypeError: string indices must be integers

In [179]:
df = pd.read_csv('store_keyword.csv')